<a href="https://colab.research.google.com/github/sarabert96/Colexification/blob/main/check/Cleaning_df_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries and data

In [2]:
import pandas
import numpy
import nltk
nltk.download('punkt')
from collections import Counter
from google.colab import files

!gdown --id 1EySPG8ZDMfMTvhUQaw8w1k6NgywI4N4S #df_all_raw


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Downloading...
From: https://drive.google.com/uc?id=1EySPG8ZDMfMTvhUQaw8w1k6NgywI4N4S
To: /content/df_all_raw.csv
244MB [00:02, 83.8MB/s]


In [3]:
from collections import Counter

In [4]:
df = pandas.read_csv('df_all_raw.csv')
df

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,dataset_ID,Form_ID,Form,clics_form,gloss_in_source,Concepticon_ID,Concepticon_Gloss,Ontological_Category,Semantic_Field,variety,Glottocode,ISO639P3code,Macroarea,Family,Latitude,Longitude,MRC_WORD,AGE_OF_ACQUISITION,CONCRETENESS,FAMILIARITY,IMAGABILITY,KUCERA_FRANCIS_FREQUENCY
0,abrahammonpa,BugunBichom-100_gold-1,san,san,gold,1369,GOLD,Person/Thing,Basic actions and technology,Bugun Bichom,bich1234,NaN,NaN,Sino-Tibetan,NaN,NaN,GOLD,NaN,576.0,550.0,594.0,52.0
1,abrahammonpa,BugunKaspi-100_gold-1,san,san,gold,1369,GOLD,Person/Thing,Basic actions and technology,Bugun Kaspi,kasp1234,NaN,NaN,Sino-Tibetan,NaN,NaN,GOLD,NaN,576.0,550.0,594.0,52.0
2,abrahammonpa,BugunNamphri-100_gold-1,san,san,gold,1369,GOLD,Person/Thing,Basic actions and technology,Bugun Namphri,namp1239,NaN,NaN,Sino-Tibetan,NaN,NaN,GOLD,NaN,576.0,550.0,594.0,52.0
3,abrahammonpa,BugunSingchung-100_gold-1,san,san,gold,1369,GOLD,Person/Thing,Basic actions and technology,Bugun Singchung,sing1271,NaN,NaN,Sino-Tibetan,NaN,NaN,GOLD,NaN,576.0,550.0,594.0,52.0
4,abrahammonpa,BugunWangho-100_gold-1,san,san,gold,1369,GOLD,Person/Thing,Basic actions and technology,Bugun Wangho,wang1301,NaN,NaN,Sino-Tibetan,NaN,NaN,GOLD,NaN,576.0,550.0,594.0,52.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1390589,zgraggenmadang,yaben-9_boy-1,waw moya,wawmoya,boy,1366,BOY,Person/Thing,Kinship,yaben,yabe1255,ybm,Papunesia,Nuclear Trans New Guinea,-4.90868,145.361,BOY,NaN,609.0,606.0,618.0,242.0
1390590,zgraggenmadang,yangulam-9_boy-1,ilaŋgum,ilanggum,boy,1366,BOY,Person/Thing,Kinship,yangulam,yang1298,ynl,Papunesia,Nuclear Trans New Guinea,-5.52697,145.858,BOY,NaN,609.0,606.0,618.0,242.0
1390591,zgraggenmadang,yangulam-9_boy-2,monaguru,monaguru,boy,1366,BOY,Person/Thing,Kinship,yangulam,yang1298,ynl,Papunesia,Nuclear Trans New Guinea,-5.52697,145.858,BOY,NaN,609.0,606.0,618.0,242.0
1390592,zgraggenmadang,yarawata-9_boy-1,wa tamana,watamana,boy,1366,BOY,Person/Thing,Kinship,yarawata,yara1250,yrw,Papunesia,Nuclear Trans New Guinea,-4.93943,145.454,BOY,NaN,609.0,606.0,618.0,242.0


1. remove diachronic varieties (middle/old/classic/ancient)

In [5]:
#tokenize variety, add flag column
df['tk_variety']=df['variety'].apply(nltk.tokenize.word_tokenize)
df

,dataset_ID,Form_ID,Form,clics_form,gloss_in_source,Concepticon_ID,Concepticon_Gloss,Ontological_Category,Semantic_Field,variety,Glottocode,ISO639P3code,Macroarea,Family,Latitude,Longitude,MRC_WORD,AGE_OF_ACQUISITION,CONCRETENESS,FAMILIARITY,IMAGABILITY,KUCERA_FRANCIS_FREQUENCY,tk_variety
0,abrahammonpa,BugunBichom-100_gold-1,san,san,gold,1369,GOLD,Person/Thing,Basic actions and technology,Bugun Bichom,bich1234,NaN,NaN,Sino-Tibetan,NaN,NaN,GOLD,NaN,576.0,550.0,594.0,52.0,"[Bugun, Bichom]"
1,abrahammonpa,BugunKaspi-100_gold-1,san,san,gold,1369,GOLD,Person/Thing,Basic actions and technology,Bugun Kaspi,kasp1234,NaN,NaN,Sino-Tibetan,NaN,NaN,GOLD,NaN,576.0,550.0,594.0,52.0,"[Bugun, Kaspi]"
2,abrahammonpa,BugunNamphri-100_gold-1,san,san,gold,1369,GOLD,Person/Thing,Basic actions and technology,Bugun Namphri,namp1239,NaN,NaN,Sino-Tibetan,NaN,NaN,GOLD,NaN,576.0,550.0,594.0,52.0,"[Bugun, Namphri]"
3,abrahammonpa,BugunSingchung-100_gold-1,san,san,gold,1369,GOLD,Person/Thing,Basic actions and technology,Bugun Singchung,sing1271,NaN,NaN,Sino-Tibetan,NaN,NaN,GOLD,NaN,576.0,550.0,594.0,52.0,"[Bugun, Singchung]"
4,abrahammonpa,BugunWangho-100_gold-1,san,san,gold,1369,GOLD,Person/Thing,Basic actions and technology,Bugun Wangho,wang1301,NaN,NaN,Sino-Tibetan,NaN,NaN,GOLD,NaN,576.0,550.0,594.0,52.0,"[Bugun, Wangho]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1390589,zgraggenmadang,yaben-9_boy-1,waw moya,wawmoya,boy,1366,BOY,Person/Thing,Kinship,yaben,yabe1255,ybm,Papunesia,Nuclear Trans New Guinea,-4.90868,145.361,BOY,NaN,609.0,606.0,618.0,242.0,[yaben]
1390590,zgraggenmadang,yangulam-9_boy-1,ilaŋgum,ilanggum,boy,1366,BOY,Person/Thing,Kinship,yangulam,yang1298,ynl,Papunesia,Nuclear Trans New Guinea,-5.52697,145.858,BOY,NaN,609.0,606.0,618.0,242.0,[yangulam]
1390591,zgraggenmadang,yangulam-9_boy-2,monaguru,monaguru,boy,1366,BOY,Person/Thing,Kinship,yangulam,yang1298,ynl,Papunesia,Nuclear Trans New Guinea,-5.52697,145.858,BOY,NaN,609.0,606.0,618.0,242.0,[yangulam]
1390592,zgraggenmadang,yarawata-9_boy-1,wa tamana,watamana,boy,1366,BOY,Person/Thing,Kinship,yarawata,yara1250,yrw,Papunesia,Nuclear Trans New Guinea,-4.93943,145.454,BOY,NaN,609.0,606.0,618.0,242.0,[yarawata]


In [6]:
df['flag']=0

In [7]:
#find non wanted varieties
diachronic_varieties = ['Old', 'Middle', 'Classic', 'Ancient', 'English']
for i in range(len(df['tk_variety'])):
  r = df['tk_variety'].iloc[i]
  if any((v in diachronic_varieties) for v in r):
    df['flag'].iloc[i]+=1


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [8]:
# delete flagged columns with non wanted categories
df= df.drop(df[df.flag > 0].index)

In [9]:
df

,dataset_ID,Form_ID,Form,clics_form,gloss_in_source,Concepticon_ID,Concepticon_Gloss,Ontological_Category,Semantic_Field,variety,Glottocode,ISO639P3code,Macroarea,Family,Latitude,Longitude,MRC_WORD,AGE_OF_ACQUISITION,CONCRETENESS,FAMILIARITY,IMAGABILITY,KUCERA_FRANCIS_FREQUENCY,tk_variety,flag
0,abrahammonpa,BugunBichom-100_gold-1,san,san,gold,1369,GOLD,Person/Thing,Basic actions and technology,Bugun Bichom,bich1234,NaN,NaN,Sino-Tibetan,NaN,NaN,GOLD,NaN,576.0,550.0,594.0,52.0,"[Bugun, Bichom]",0
1,abrahammonpa,BugunKaspi-100_gold-1,san,san,gold,1369,GOLD,Person/Thing,Basic actions and technology,Bugun Kaspi,kasp1234,NaN,NaN,Sino-Tibetan,NaN,NaN,GOLD,NaN,576.0,550.0,594.0,52.0,"[Bugun, Kaspi]",0
2,abrahammonpa,BugunNamphri-100_gold-1,san,san,gold,1369,GOLD,Person/Thing,Basic actions and technology,Bugun Namphri,namp1239,NaN,NaN,Sino-Tibetan,NaN,NaN,GOLD,NaN,576.0,550.0,594.0,52.0,"[Bugun, Namphri]",0
3,abrahammonpa,BugunSingchung-100_gold-1,san,san,gold,1369,GOLD,Person/Thing,Basic actions and technology,Bugun Singchung,sing1271,NaN,NaN,Sino-Tibetan,NaN,NaN,GOLD,NaN,576.0,550.0,594.0,52.0,"[Bugun, Singchung]",0
4,abrahammonpa,BugunWangho-100_gold-1,san,san,gold,1369,GOLD,Person/Thing,Basic actions and technology,Bugun Wangho,wang1301,NaN,NaN,Sino-Tibetan,NaN,NaN,GOLD,NaN,576.0,550.0,594.0,52.0,"[Bugun, Wangho]",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1390589,zgraggenmadang,yaben-9_boy-1,waw moya,wawmoya,boy,1366,BOY,Person/Thing,Kinship,yaben,yabe1255,ybm,Papunesia,Nuclear Trans New Guinea,-4.90868,145.361,BOY,NaN,609.0,606.0,618.0,242.0,[yaben],0
1390590,zgraggenmadang,yangulam-9_boy-1,ilaŋgum,ilanggum,boy,1366,BOY,Person/Thing,Kinship,yangulam,yang1298,ynl,Papunesia,Nuclear Trans New Guinea,-5.52697,145.858,BOY,NaN,609.0,606.0,618.0,242.0,[yangulam],0
1390591,zgraggenmadang,yangulam-9_boy-2,monaguru,monaguru,boy,1366,BOY,Person/Thing,Kinship,yangulam,yang1298,ynl,Papunesia,Nuclear Trans New Guinea,-5.52697,145.858,BOY,NaN,609.0,606.0,618.0,242.0,[yangulam],0
1390592,zgraggenmadang,yarawata-9_boy-1,wa tamana,watamana,boy,1366,BOY,Person/Thing,Kinship,yarawata,yara1250,yrw,Papunesia,Nuclear Trans New Guinea,-4.93943,145.454,BOY,NaN,609.0,606.0,618.0,242.0,[yarawata],0


Now we have the original dataframe removed from unwanted varieties

In [21]:
# modify column with word count: flag > 0 if multiword
df['flag'] = df['Concepticon_Gloss'].str.count(' OR ')
df['flag'] = df['Concepticon_Gloss'].str.count("\(")

In [22]:
df[df['flag']==1]

,dataset_ID,Form_ID,Form,clics_form,gloss_in_source,Concepticon_ID,Concepticon_Gloss,Ontological_Category,Semantic_Field,variety,Glottocode,ISO639P3code,Macroarea,Family,Latitude,Longitude,MRC_WORD,AGE_OF_ACQUISITION,CONCRETENESS,FAMILIARITY,IMAGABILITY,KUCERA_FRANCIS_FREQUENCY,tk_variety,flag
92,abrahammonpa,BugunBichom-103_hair-1,khazijaŋ,khazijang,hair,2648,HAIR (HEAD),Person/Thing,The body,Bugun Bichom,bich1234,NaN,NaN,Sino-Tibetan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Bugun, Bichom]",1
93,abrahammonpa,BugunKaspi-103_hair-1,khazijaŋ,khazijang,hair,2648,HAIR (HEAD),Person/Thing,The body,Bugun Kaspi,kasp1234,NaN,NaN,Sino-Tibetan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Bugun, Kaspi]",1
94,abrahammonpa,BugunNamphri-103_hair-1,khazijaŋ,khazijang,hair,2648,HAIR (HEAD),Person/Thing,The body,Bugun Namphri,namp1239,NaN,NaN,Sino-Tibetan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Bugun, Namphri]",1
95,abrahammonpa,BugunSingchung-103_hair-1,khazijaŋ,khazijang,hair,2648,HAIR (HEAD),Person/Thing,The body,Bugun Singchung,sing1271,NaN,NaN,Sino-Tibetan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Bugun, Singchung]",1
96,abrahammonpa,BugunWangho-103_hair-1,khazijau,khazijau,hair,2648,HAIR (HEAD),Person/Thing,The body,Bugun Wangho,wang1301,NaN,NaN,Sino-Tibetan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Bugun, Wangho]",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1389766,zgraggenmadang,yaben-89_excrement-1,i:bi,ibi,excrement,675,FAECES (EXCREMENT),Person/Thing,The body,yaben,yabe1255,ybm,Papunesia,Nuclear Trans New Guinea,-4.90868,145.3610,NaN,NaN,NaN,NaN,NaN,NaN,[yaben],1
1389767,zgraggenmadang,yabong-89_excrement-1,sim,sim,excrement,675,FAECES (EXCREMENT),Person/Thing,The body,yabong,yabo1240,ybo,Papunesia,Nuclear Trans New Guinea,-5.61718,146.2065,NaN,NaN,NaN,NaN,NaN,NaN,[yabong],1
1389768,zgraggenmadang,yangulam-89_excrement-1,simbi,simbi,excrement,675,FAECES (EXCREMENT),Person/Thing,The body,yangulam,yang1298,ynl,Papunesia,Nuclear Trans New Guinea,-5.52697,145.8580,NaN,NaN,NaN,NaN,NaN,NaN,[yangulam],1
1389769,zgraggenmadang,yarawata-89_excrement-1,ibia,ibia,excrement,675,FAECES (EXCREMENT),Person/Thing,The body,yarawata,yara1250,yrw,Papunesia,Nuclear Trans New Guinea,-4.93943,145.4540,NaN,NaN,NaN,NaN,NaN,NaN,[yarawata],1


In [26]:
df.shape

(1358416, 24)

In [27]:
# delete flagged columns
df = df.drop(df[df.flag > 0].index)
# delete column with word count
del df['flag']

In [28]:
df.shape

(1238663, 23)

df is now without bad concepts (OR, bracket)

In [42]:
df['Concepticon_Gloss'].isna().values.any()

False

In [44]:
# reset index
df = df.reset_index(drop=True)

Save df on csv

In [45]:
df.to_csv('df_all_ok.csv')